In [10]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, mean_squared_error
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scipy.stats import ttest_rel
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import model_selection
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import matplotlib.pyplot as plt

# Load data

In [11]:

# Bootstrapping function
def bootstrap(data, n_samples):
    np.random.seed(0)  # For reproducibility
    indices = np.random.choice(data.index, size=n_samples, replace=True)
    return data.iloc[indices]

# Example usage:
# n_samples = len(data)  # Typically, the same number of samples as the original dataset
# bootstrapped_data = bootstrap(data, n_samples)

# Now we can use bootstrapped_data for modeling instead of the original data


In [14]:
data = pd.read_csv('/Users/mikelyu/Desktop/Uni/02455 Statistical Evaluation/02455-projects/HR_data.csv')

# Display the first few rows of the dataset
print(data.head())

   Unnamed: 0    HR_Mean  HR_Median    HR_std  HR_Min  HR_Max     HR_AUC  \
0           0  77.965186      78.00  3.345290   73.23   83.37  22924.945   
1           1  70.981097      70.57  2.517879   67.12   78.22  21930.400   
2           2  73.371959      73.36  3.259569   67.88   80.22  21647.085   
3           3  78.916822      77.88  4.054595   72.32   84.92  25258.905   
4           4  77.322226      74.55  6.047603   70.52   90.15  23890.565   

     Round   Phase  Individual  Puzzler  Frustrated Cohort  
0  round_3  phase3           1        1           1   D1_1  
1  round_3  phase2           1        1           5   D1_1  
2  round_3  phase1           1        1           0   D1_1  
3  round_2  phase3           1        1           1   D1_1  
4  round_2  phase2           1        1           5   D1_1  


In [13]:
# Features and target
X = bootstrapped_data[['HR_Mean', 'HR_Median', 'HR_std', 'HR_Min', 'HR_Max', 'HR_AUC']]
y = bootstrapped_data['Frustrated']
groups = bootstrapped_data['Individual']

# Model
model = RandomForestClassifier()

# GroupKFold with n_splits=14 (equivalent to Leave-One-Group-Out)
group_kfold = GroupKFold(n_splits=14)

# Cross-Validation Scores
cv_results = {
    "accuracy": [],
    "precision": [],
    "recall": [],
    "f1": [],
    "roc_auc": []
}

for train_idx, test_idx in group_kfold.split(X, y, groups):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]
    
    cv_results['accuracy'].append(accuracy_score(y_test, y_pred))
    cv_results['precision'].append(precision_score(y_test, y_pred, average='weighted'))
    cv_results['recall'].append(recall_score(y_test, y_pred, average='weighted'))
    cv_results['f1'].append(f1_score(y_test, y_pred, average='weighted'))
    cv_results['roc_auc'].append(roc_auc_score(y_test, y_prob, multi_class='ovr'))

# Display Cross-Validation Scores
for metric in cv_results:
    print(f"{metric.capitalize()} Scores: {cv_results[metric]}")
    print(f"Mean {metric.capitalize()} Score: {np.mean(cv_results[metric])}")

# Box plot for visualizing the cross-validation scores
plt.figure(figsize=(10, 6))
plt.boxplot(cv_results.values(), labels=cv_results.keys())
plt.title('Cross-Validation Scores')
plt.ylabel('Score')
plt.show()


NameError: name 'bootstrapped_data' is not defined